- gungu, dong 으로 segment -> test에만 있는 값이 있는지 확인

# Library Setting

In [1]:
import sys
sys.path.append('/Volumes/KHJ/Github/hyuckjinkim/lib-python')

from base import gc_collect_all, setdiff
from filesystem_utils import mkdir
from graph import abline
from data_prepare import (
    get_holiday, reduce_mem_usage, delete_unique_columns,
    TypeController, CategoricalQuantileCalculator,
    GroupScaler, OneHotEncoder, InteractionTerm, TargetTransform,
)

In [2]:
gc_collect_all()

In [3]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

from tqdm import tqdm
tqdm.pandas()

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('mode.chained_assignment', None)

import datetime
import glob

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

In [4]:
def return_unique_columns(data):
    unique_info = data.nunique()
    unique_cols = unique_info[unique_info==1].index.tolist()
    return unique_cols

In [5]:
class CFG:
    SEED = 42
    TARGET = ['사망자수','중상자수','경상자수','부상자수']
    TARGET_TRANSFORMATION = 'identity'
    N_SPLITS = 5

<br></br>

# Data

## Data Load

In [6]:
train_df = pd.read_csv('./data/train.csv')
test_df  = pd.read_csv('./data/test.csv')

# train_additional_df = pd.read_csv('./data/external_open/countrywide_accident.csv',encoding='utf-8')
# cctv_df = pd.read_csv('./data/external_open/대구 CCTV 정보.csv',encoding='cp949')
# security_light_df = pd.read_csv('./data/external_open/대구 보안등 정보.csv',encoding='cp949',low_memory=False)
# child_zone_df = pd.read_csv('./data/external_open/대구 어린이 보호 구역 정보.csv',encoding='cp949')
# parking_df = pd.read_csv('./data/external_open/대구 주차장 정보.csv',encoding='cp949')

In [7]:
display(train_df.head(1))
display(train_df.tail(1))

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO
0,ACCIDENT_00000,2019-01-01 00,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,승용,여,51세,상해없음,보행자,여,70세,중상,0,1,0,0,5


,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO
39608,ACCIDENT_39608,2021-12-31 23,금요일,맑음,대구광역시 서구 비산동,단일로 - 지하차도(도로)내,건조,차대차,측면충돌,안전운전불이행,승용,남,27세,상해없음,승용,남,33세,경상,0,0,2,0,6


<br>

## Preprocessing

In [8]:
def add_week(weekend):
    week = 1
    week_list = []
    for weekend in weekend:
        if weekend==1:
            week+=1
        week_list.append(week)
    return week_list

def split_location(location):
    error_return = [np.nan,np.nan]
    if location is np.nan:
        return error_return
    sido, gungu, dong = location.split(' ')
    dong = dong.split('동')[0] + '동'
    if sido not in ['대구광역시','대구']:
        print(f"{sido}: sido is not '대구광역시'.")
        return error_return
    if gungu[-1:] not in ['구','군']:
        print(f"{gungu}: gungu is not '~구' or '~군'")
        return error_return
    return [gungu,dong]

def preprocessing(data):
    d = data.copy()
    
    # (1) test data에 없는 컬럼 제거
    no_columns_in_test = ['사고유형 - 세부분류','법규위반','가해운전자 차종','가해운전자 성별','가해운전자 연령',
                          '가해운전자 상해정도','피해운전자 차종','피해운전자 성별','피해운전자 연령','피해운전자 상해정도']
    no_columns_in_test = list(set(d.columns)&set(no_columns_in_test))
    d.drop(no_columns_in_test,axis=1,inplace=True)
    
    # (1) 시군구: 시/군구/동 -> 군구/동
    d[['gungu','dong']] = np.stack(d['시군구'].apply(split_location))
    d.drop('시군구',axis=1,inplace=True)
    
    # (2) 도로형태
    d[['roadtype','roadtype_detail']] = np.stack(d['도로형태'].str.split(' - '))
    d['is_parking'] = d['roadtype'].map({'단일로':0,'교차로':0,'기타':np.nan,'주차장':1,'미분류':np.nan})
    d['is_road']    = d['roadtype'].map({'단일로':1,'교차로':1,'기타':np.nan,'주차장':np.nan,'미분류':np.nan})
    d.drop('도로형태',axis=1,inplace=True)
    for col in ['is_parking','is_road']:
        d[col] = d[col].fillna(0)

    return d

def feature_engineering(data):
    d = data.copy()
    date = d['사고일시'].apply(lambda x: datetime.datetime.strptime(str(x),'%Y-%m-%d %H'))
    
    # (1) date columns
    d['year']       = date.dt.year
    d['month']      = date.dt.month
    d['day']        = date.dt.day
    d['hour']       = date.dt.hour
    d['weekday']    = date.dt.weekday
    d['weekend']    = date.dt.weekday.isin([5,6]).astype(int)
    d['week']       = add_week(d['weekend'])

    # (2) is holiday & is dayoff
    holiday_list = get_holiday(d['year'].unique())
    d['is_holiday'] = date.isin(holiday_list).astype(int)
    d['is_dayoff']  = ((d.is_holiday==1) | (d.weekend==1)).astype(int)
    
    # (3) unuse features
    unuse_features = ['ID','사고일시','요일','ECLO'] #'사망자수','중상자수','경상자수','부상자수'
    unuse_features = list(set(d.columns)&set(unuse_features))
    d.drop(columns=unuse_features,inplace=True)
    
    return d

In [9]:
train_df = preprocessing(train_df)
train_df = feature_engineering(train_df)

test_df = preprocessing(test_df)
test_df = feature_engineering(test_df)

In [10]:
train_df.head()

,기상상태,노면상태,사고유형,사망자수,중상자수,경상자수,부상자수,gungu,dong,roadtype,roadtype_detail,is_parking,is_road,year,month,day,hour,weekday,weekend,week,is_holiday,is_dayoff
0,맑음,건조,차대사람,0,1,0,0,중구,대신동,단일로,기타,0.0,1.0,2019,1,1,0,1,0,1,1,1
1,흐림,건조,차대사람,0,0,1,0,달서구,감삼동,단일로,기타,0.0,1.0,2019,1,1,0,1,0,1,1,1
2,맑음,건조,차대사람,0,0,1,0,수성구,두산동,단일로,기타,0.0,1.0,2019,1,1,1,1,0,1,0,0
3,맑음,건조,차대차,0,1,0,0,북구,복현동,단일로,기타,0.0,1.0,2019,1,1,2,1,0,1,0,0
4,맑음,건조,차대차,0,0,1,0,동구,신암동,단일로,기타,0.0,1.0,2019,1,1,4,1,0,1,0,0


<br>

## Merge Addtional Information

In [11]:
def add_cctv_info(data):
    d = data.copy()

    cctv_df = pd.read_csv('./data/external_open/대구 CCTV 정보.csv',encoding='cp949')

    # 다사읍 -> 다사읍동
    cctv_df['소재지지번주소'] = cctv_df['소재지지번주소'].str.replace(' 다사읍 ',' 다사읍동 ')

    # 수기로 수정 (1)
    raw = ['대구광역시 다사읍동 세천리 1684-4','대구광역시 가창면 삼산리 산 327-9']
    new = ['대구광역시 달성군 다사읍동 세천리 1684-4','대구광역시 달성군 가창면 삼산리 산 327-9']
    for r,n in zip(raw,new):
        loc = cctv_df['소재지지번주소'] == r
        cctv_df.loc[loc, '소재지지번주소'] = n

    # 수기로 수정 (2)
    raw = ['대구광역시 중구 종로 17','대구광역시 중구 국채보상로 713']
    new = ['대구 중구 남성로동 92','대구 중구 동인동4가 89-1']
    for r,n in zip(raw,new):
        loc = (cctv_df['소재지지번주소'].isnull()) & (cctv_df['소재지도로명주소']==r)
        cctv_df.loc[loc, '소재지지번주소'] = n

    cctv_df[['gungu','dong']] = [[np.nan,np.nan] if address is np.nan else
                                 split_location(' '.join(address.split(' ')[:3]))
                                 for address in cctv_df['소재지지번주소']]

    # 단속구분별 cctv 수
    for v in [1,2,4]:
        colname = f'n_cctv_단속구분{v}'
        sub = cctv_df[cctv_df['단속구분']==v]
        n_cctv = sub.groupby(['gungu','dong']).size().reset_index().rename(columns={0:colname})
        d = pd.merge(d,n_cctv,how='left',on=['gungu','dong'])
        d[colname].fillna(0,inplace=True)

    # 도로노선방향별 cctv 수
    for v in [1,2,3]:
        colname = f'n_cctv_도로노선방향{v}'
        sub = cctv_df[cctv_df['도로노선방향']==v]
        n_cctv = sub.groupby(['gungu','dong']).size().reset_index().rename(columns={0:colname})
        d = pd.merge(d,n_cctv,how='left',on=['gungu','dong'])
        d[colname].fillna(0,inplace=True)

    # 'n_cctv_도로노선방향3', 'n_cctv_단속구분4'는 correlation이 100%임
    d.drop('n_cctv_도로노선방향3',axis=1,inplace=True)

    # 제한속도 최소/최대/평균
    limit_speed = cctv_df\
        .groupby(['gungu','dong'])['제한속도']\
        .agg(min=np.nanmin,max=np.nanmax,avg=np.nanmean)\
        .add_prefix('limit_speed_')\
        .reset_index()
    d = pd.merge(d,limit_speed,how='left',on=['gungu','dong'])

    # 설치연도 최소/최대/평균
    installation_year = cctv_df\
        .groupby(['gungu','dong'])['설치연도']\
        .agg(min=np.nanmin,max=np.nanmax,avg=np.nanmean)\
        .add_prefix('installation_year_')\
        .reset_index()
    d = pd.merge(d,installation_year,how='left',on=['gungu','dong'])

    # 보호구역구분: 1,2인 경우 보호구역으로 보고, 나머지(99,nan)는 보호구역이 아닌 것으로 봄
    cctv_df['is_protectzone'] = np.where(cctv_df['보호구역구분'].isin([1,2]),1,0)
    protectzone = cctv_df.groupby(['gungu','dong'])['is_protectzone'].agg(n_protectzone=np.nansum).reset_index()
    d = pd.merge(d,protectzone,how='left',on=['gungu','dong'])
    
    return d

In [12]:
train_df = add_cctv_info(train_df)
test_df  = add_cctv_info(test_df)

<br>

## Target Transformation

In [13]:
target_transform = TargetTransform(func=CFG.TARGET_TRANSFORMATION, offset=1)
train_df[CFG.TARGET] = target_transform.fit_transform(
    target=train_df[CFG.TARGET],
)

In [14]:
train_df.head()

,기상상태,노면상태,사고유형,사망자수,중상자수,경상자수,부상자수,gungu,dong,roadtype,roadtype_detail,is_parking,is_road,year,month,day,hour,weekday,weekend,week,is_holiday,is_dayoff,n_cctv_단속구분1,n_cctv_단속구분2,n_cctv_단속구분4,n_cctv_도로노선방향1,n_cctv_도로노선방향2,limit_speed_min,limit_speed_max,limit_speed_avg,installation_year_min,installation_year_max,installation_year_avg,n_protectzone
0,맑음,건조,차대사람,0.0,1.0,0.0,0.0,중구,대신동,단일로,기타,0.0,1.0,2019,1,1,0,1,0,1,1,1,0.0,1.0,4.0,0.0,1.0,0.0,40.0,8.0,2006.0,2021.0,2013.000000,2.0
1,흐림,건조,차대사람,0.0,0.0,1.0,0.0,달서구,감삼동,단일로,기타,0.0,1.0,2019,1,1,0,1,0,1,1,1,1.0,3.0,8.0,3.0,1.0,0.0,60.0,17.5,2008.0,2021.0,2015.333333,2.0
2,맑음,건조,차대사람,0.0,0.0,1.0,0.0,수성구,두산동,단일로,기타,0.0,1.0,2019,1,1,1,1,0,1,0,0,2.0,2.0,0.0,3.0,1.0,50.0,60.0,55.0,2014.0,2021.0,2018.250000,0.0
3,맑음,건조,차대차,0.0,1.0,0.0,0.0,북구,복현동,단일로,기타,0.0,1.0,2019,1,1,2,1,0,1,0,0,2.0,8.0,0.0,7.0,3.0,30.0,50.0,38.0,2014.0,2021.0,2018.300000,4.0
4,맑음,건조,차대차,0.0,0.0,1.0,0.0,동구,신암동,단일로,기타,0.0,1.0,2019,1,1,4,1,0,1,0,0,2.0,10.0,0.0,9.0,3.0,30.0,50.0,40.0,2016.0,2020.0,2018.250000,7.0


<br>

## Quantile values of Target

In [15]:
cat_features = ['기상상태','노면상태','사고유형','gungu','dong','roadtype','roadtype_detail']

In [16]:
# feature engineering
calculator = CategoricalQuantileCalculator()
calculator.fit(
    data=train_df,
    test_data=test_df,
    target_feature=CFG.TARGET,
    cat_features=cat_features,
    subset_depth=3,#len(cat_features),
)
train_df = calculator.transform(train_df)
test_df  = calculator.transform(test_df)

> Get quantiles of target by categorical features (depth=3)


Subset: dong + roadtype + roadtype_detail: 100%|█| 63/63 [00:08<00:00,  7.61it/s


<br>

## Group Scaler

In [17]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = GroupScaler(scaler=MinMaxScaler())
# scaler.fit(
#     data=train_df,
#     group=CFG.KEY_FEATURES,
#     num_features=num_features,
# )
# train_df = scaler.transform(train_df)
# test_df  = scaler.transform(test_df)

<br>

## Memory Reduce

In [18]:
train_df, _ = reduce_mem_usage(train_df,verbose=False)
test_df , _ = reduce_mem_usage(test_df ,verbose=False)

<br></br>

# Modeling

In [19]:
# https://dacon.io/en/codeshare/6499

In [20]:
# https://stackoverflow.com/questions/66785587/how-do-i-use-validation-sets-on-multioutputregressor-for-xgbregressor

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from sklearn.multioutput import MultiOutputRegressor
from sklearn.multioutput import _fit_estimator
from sklearn.utils.validation import _check_fit_params
from sklearn.base import is_classifier
from sklearn.utils.fixes import delayed
from joblib import Parallel

class CustomMultiOutputRegressor(MultiOutputRegressor):
    
    def fit(self, X, y, sample_weight=None, **fit_params):
        """ Fit the model to data.
        Fit a separate model for each output variable.
        Parameters
        ----------
        X : {array-like, sparse matrix} of shape (n_samples, n_features)
            Data.
        y : {array-like, sparse matrix} of shape (n_samples, n_outputs)
            Multi-output targets. An indicator matrix turns on multilabel
            estimation.
        sample_weight : array-like of shape (n_samples,), default=None
            Sample weights. If None, then samples are equally weighted.
            Only supported if the underlying regressor supports sample
            weights.
        **fit_params : dict of string -> object
            Parameters passed to the ``estimator.fit`` method of each step.
            .. versionadded:: 0.23
        Returns
        -------
        self : object
        """

        if not hasattr(self.estimator, "fit"):
            raise ValueError("The base estimator should implement"
                             " a fit method")

        X, y = self._validate_data(X, y,
                                   force_all_finite=False,
                                   multi_output=True, accept_sparse=True)

        if is_classifier(self):
            check_classification_targets(y)

        if y.ndim == 1:
            raise ValueError("y must have at least two dimensions for "
                             "multi-output regression but has only one.")

        if (sample_weight is not None and
                not has_fit_parameter(self.estimator, 'sample_weight')):
            raise ValueError("Underlying estimator does not support"
                             " sample weights.")

        fit_params_validated = _check_fit_params(X, fit_params)
            
        self.estimators_ = Parallel(n_jobs=self.n_jobs)(
            delayed(_fit_estimator)(
                self.estimator, np.array(X), y[:, i], sample_weight,
                **fit_params_validated,
            ) 
            for i in range(y.shape[1])
        )
        return self

class CustomEvalsetMultiOutputRegressor(MultiOutputRegressor):
    
    def fit(self, X, y, sample_weight=None, **fit_params):
        """ Fit the model to data.
        Fit a separate model for each output variable.
        Parameters
        ----------
        X : {array-like, sparse matrix} of shape (n_samples, n_features)
            Data.
        y : {array-like, sparse matrix} of shape (n_samples, n_outputs)
            Multi-output targets. An indicator matrix turns on multilabel
            estimation.
        sample_weight : array-like of shape (n_samples,), default=None
            Sample weights. If None, then samples are equally weighted.
            Only supported if the underlying regressor supports sample
            weights.
        **fit_params : dict of string -> object
            Parameters passed to the ``estimator.fit`` method of each step.
            .. versionadded:: 0.23
        Returns
        -------
        self : object
        """

        if not hasattr(self.estimator, "fit"):
            raise ValueError("The base estimator should implement"
                             " a fit method")

        X, y = self._validate_data(X, y,
                                   force_all_finite=False,
                                   multi_output=True, accept_sparse=True)

        if is_classifier(self):
            check_classification_targets(y)

        if y.ndim == 1:
            raise ValueError("y must have at least two dimensions for "
                             "multi-output regression but has only one.")

        if (sample_weight is not None and
                not has_fit_parameter(self.estimator, 'sample_weight')):
            raise ValueError("Underlying estimator does not support"
                             " sample weights.")

        fit_params_validated = _check_fit_params(X, fit_params)
        [(X_test, Y_test)] = fit_params_validated.pop('eval_set')
            
        self.estimators_ = Parallel(n_jobs=self.n_jobs)(
            delayed(_fit_estimator)(
                self.estimator, np.array(X), y[:, i], sample_weight,
                **fit_params_validated,
                eval_set=[(np.array(X_test), np.array(Y_test)[:, i])] #.iloc로 수정
            ) 
            for i in range(y.shape[1])
        )
        return self
    
class MultiOutputLGBMRegressor(CustomEvalsetMultiOutputRegressor):
    def __init__(self, estimator):
        super().__init__(estimator)

class MultiOutputXGBRegressor(CustomEvalsetMultiOutputRegressor):
    def __init__(self, estimator):
        super().__init__(estimator)
        
class MultiOutputRandomForestRegressor(CustomMultiOutputRegressor):
    def __init__(self, estimator):
        super().__init__(estimator)

class MultiOutputHistGradientBoostingRegressor(CustomMultiOutputRegressor):
    def __init__(self, estimator):
        super().__init__(estimator)

In [51]:
from copy import deepcopy
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.base import BaseEstimator, RegressorMixin
from catboost import CatBoostRegressor, Pool
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
import time
import pickle
import dill

def RMSE(y_true, y_pred):
    return mean_squared_error(y_true=y_true,y_pred=y_pred)**0.5

def WeightedMultiRMSE(y_true, y_pred, weight):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    weight = np.array(weight)
    assert y_true.shape==y_pred.shape, \
        "prediction shape is not correct. y_true: {}, y_pred: {}".format(y_true.shape,y_pred.shape)
    assert y_true.shape[1]==len(weight), \
        "weight shape is not correct. y_true: {}, weight: {}".format(y_true.shape,len(weight))
    if sum(weight)!=1:
        weight = weight / sum(weight)
    
    rmse_list = []
    for k in range(y_true.shape[1]):
        rmse = mean_squared_error(y_true[:,k],y_pred[:,k])**0.5
        rmse_list.append(rmse)

    return rmse_list @ weight

def WeightedMultiRMSLE(y_true, y_pred, weight):
    return WeightedMultiRMSE(np.log1p(y_true),np.log1p(y_pred),weight)
    
def RMSLE(y_true, y_pred):
    log_true = np.log1p(y_true)
    log_pred = np.log1p(y_pred)
    squared_error = (log_true-log_pred)**2
    return np.sqrt(np.mean(squared_error))

In [22]:
# 2) 업데이트버전
#  (1) stacking 추가
#  (2) LGBM에 sample_weight 추가
class WeightedEnsembleRegressor(BaseEstimator, RegressorMixin):
    def __init__(self,
                 hyperparameters,
                 weight='balanced',
                 inverse_transform=None,
                 eval_metric=None,
                 method='ensemble',
                 use_weightedsum_in_stacking=True):
        super().__init__()
        
        assert weight in ['equal','balanced'], \
            "weight must be one of ['equal','balanced']"
        
        self.hyperparameters = hyperparameters
        self.weight = weight
        self.inverse_transform = inverse_transform
        self.eval_metric = RMSE if eval_metric is None else eval_metric
        self.method = method
        self.use_weightedsum_in_stacking = use_weightedsum_in_stacking
        
        self._get_regressors()
        self._get_regressors_name()
        
        if use_weightedsum_in_stacking:
            self.stacking_feature = 'pred'
        else:
            self.stacking_feature = [f'pred{i+1}' for i in range(len(self.regressors))]
            
    def _get_regressors(self):
        max_depth = 9
        n_jobs = -1
        cat_loss_function = 'MultiRMSE' # 'RMSE','MAE'
        cat_eval_metric = 'MultiRMSE'   # 'RMSE','MAE'
        lgb_metric = 'rmse' # 'rmse','mean_absolute_error'
        
        params_ridge = {
            'alphas' : [1e-5, 1e-3, 1e-1, 1.0, 10.0, 100.0],
            'cv' : RepeatedKFold(n_splits=5, n_repeats=3, 
                                 random_state=self.hyperparameters['random_state']),
        }
        
        params_lasso = {
            'alphas' : [1e-5, 1e-3, 1e-1, 1.0, 10.0, 100.0],
            'cv' : RepeatedKFold(n_splits=5, n_repeats=3,
                                 random_state=self.hyperparameters['random_state']),
            'n_jobs' : n_jobs,
            #'max_iter' : 30000,
            'tol' : 0.001,
        }
        
        params_elasticnet = {
            'l1_ratio' : np.arange(0.1, 1, 0.1),
            'alphas' : [1e-5, 1e-3, 1e-1, 1.0, 10.0, 100.0],
            'cv' : RepeatedKFold(n_splits=5, n_repeats=3,
                                 random_state=self.hyperparameters['random_state']),
            'n_jobs' : n_jobs,
            #'max_iter' : 30000,
            'tol' : 0.001,
        }
        
        params_catboost1 = {
            'random_state' : self.hyperparameters['random_state'],
            'iterations' : self.hyperparameters['iterations'],
            'early_stopping_rounds' : self.hyperparameters['early_stopping_rounds'],
            'learning_rate' : self.hyperparameters['learning_rate'],
            'loss_function' : cat_loss_function,
            #'loss_function' : cat_loss_function, 'eval_metric' : cat_eval_metric,
            'grow_policy' : 'Lossguide', # 'SymmetricTree','Depthwise'
            'use_best_model' : True,
            'allow_writing_files' : False,
            'verbose' : 0,
            'max_depth' : self.hyperparameters['max_depth'],
            #'l2_leaf_reg' : 1,
        }
        
        params_catboost2 = {
            'random_state' : self.hyperparameters['random_state'],
            'iterations' : self.hyperparameters['iterations'],
            'early_stopping_rounds' : self.hyperparameters['early_stopping_rounds'],
            'learning_rate' : self.hyperparameters['learning_rate'],
            'loss_function' : cat_loss_function,
            #'loss_function' : cat_loss_function, 'eval_metric' : cat_eval_metric,
            'grow_policy' : 'Lossguide', # 'SymmetricTree','Depthwise'
            'use_best_model' : True,
            'allow_writing_files' : False,
            'verbose' : 0,
            #'max_depth' : self.hyperparameters['max_depth'],
            'l2_leaf_reg' : 3,
        }
        
        params_catboost3 = {
            'random_state' : self.hyperparameters['random_state'],
            'iterations' : self.hyperparameters['iterations'],
            'early_stopping_rounds' : self.hyperparameters['early_stopping_rounds'],
            'learning_rate' : self.hyperparameters['learning_rate'],
            'loss_function' : cat_loss_function,
            #'loss_function' : cat_loss_function, 'eval_metric' : cat_eval_metric,
            'grow_policy' : 'SymmetricTree', # 'Lossguide','Depthwise'
            'use_best_model' : True,
            'allow_writing_files' : False,
            'verbose' : 0,
            #'max_depth' : self.hyperparameters['max_depth'],
            'l2_leaf_reg' : 1,
        }
        
        params_catboost4 = {
            'random_state' : self.hyperparameters['random_state'],
            'iterations' : self.hyperparameters['iterations'],
            'early_stopping_rounds' : self.hyperparameters['early_stopping_rounds'],
            'learning_rate' : self.hyperparameters['learning_rate'],
            'loss_function' : cat_loss_function,
            #'loss_function' : cat_loss_function, 'eval_metric' : cat_eval_metric,
            'grow_policy' : 'Depthwise', # 'SymmetricTree','Depthwise'
            'use_best_model' : True,
            'allow_writing_files' : False,
            'verbose' : 0,
            'max_depth' : self.hyperparameters['max_depth'],
            'l2_leaf_reg' : 1,
        }
        
        params_lightgbm1 = {
            'random_state' : self.hyperparameters['random_state'],
            'n_estimators' : self.hyperparameters['iterations'],
            'early_stopping_round' : self.hyperparameters['early_stopping_rounds'],
            'learning_rate' : self.hyperparameters['learning_rate'],
            'objective' : 'regression',
            'metric' : lgb_metric,
            'verbosity' : -1,
            'max_depth': self.hyperparameters['max_depth'],
            'reg_lambda' : 1,
            'n_jobs' : n_jobs,
        }
        
        params_lightgbm2 = {
            'random_state' : self.hyperparameters['random_state'],
            'n_estimators' : self.hyperparameters['iterations'],
            'early_stopping_round' : self.hyperparameters['early_stopping_rounds'],
            'learning_rate' : self.hyperparameters['learning_rate'],
            'objective' : 'regression',
            'metric' : lgb_metric,
            'verbosity' : -1,
            'max_depth': self.hyperparameters['max_depth'],
            'reg_lambda' : 3,
            'n_jobs' : n_jobs,
        }
        
        params_lightgbm3 = {
            'random_state' : self.hyperparameters['random_state'],
            'n_estimators' : self.hyperparameters['iterations'],
            'early_stopping_round' : self.hyperparameters['early_stopping_rounds'],
            'learning_rate' : self.hyperparameters['learning_rate'],
            'objective' : 'regression',
            'metric' : lgb_metric,
            'verbosity' : -1,
            'max_depth': self.hyperparameters['max_depth'],
            'reg_alpha' : 1,
            'n_jobs' : n_jobs,
        }
        
        params_lightgbm4 = {
            'random_state' : self.hyperparameters['random_state'],
            'n_estimators' : self.hyperparameters['iterations'],
            'early_stopping_round' : self.hyperparameters['early_stopping_rounds'],
            'learning_rate' : self.hyperparameters['learning_rate'],
            'objective' : 'regression',
            'metric' : lgb_metric,
            'verbosity' : -1,
            'max_depth': self.hyperparameters['max_depth'],
            'reg_alpha' : 3,
            'n_jobs' : n_jobs,
        }
        
        params_lightgbm5 = {
            'random_state' : self.hyperparameters['random_state'],
            'n_estimators' : self.hyperparameters['iterations'],
            'early_stopping_round' : self.hyperparameters['early_stopping_rounds'],
            'learning_rate' : self.hyperparameters['learning_rate'],
            'objective' : 'regression',
            'metric' : lgb_metric,
            'verbosity' : -1,
            'max_depth': self.hyperparameters['max_depth'],
            'reg_alpha' : 1,
            'reg_lambda' : 1,
            'n_jobs' : n_jobs,
        }
        
        params_lightgbm6 = {
            'random_state' : self.hyperparameters['random_state'],
            'n_estimators' : self.hyperparameters['iterations'],
            'early_stopping_round' : self.hyperparameters['early_stopping_rounds'],
            'learning_rate' : self.hyperparameters['learning_rate'],
            'objective' : 'regression',
            'metric' : lgb_metric,
            'verbosity' : -1,
            'max_depth': self.hyperparameters['max_depth'],
            'reg_alpha' : 1,
            'reg_lambda' : 3,
            'n_jobs' : n_jobs,
        }
        
        params_lightgbm7 = {
            'random_state' : self.hyperparameters['random_state'],
            'n_estimators' : self.hyperparameters['iterations'],
            'early_stopping_round' : self.hyperparameters['early_stopping_rounds'],
            'learning_rate' : self.hyperparameters['learning_rate'],
            'objective' : 'regression',
            'metric' : lgb_metric,
            'verbosity' : -1,
            'max_depth': self.hyperparameters['max_depth'],
            'reg_alpha' : 3,
            'reg_lambda' : 1,
            'n_jobs' : n_jobs,
        }
        
        params_lightgbm8 = {
            'random_state' : self.hyperparameters['random_state'],
            'n_estimators' : self.hyperparameters['iterations'],
            'early_stopping_round' : self.hyperparameters['early_stopping_rounds'],
            'learning_rate' : self.hyperparameters['learning_rate'],
            'objective' : 'regression',
            'metric' : lgb_metric,
            'verbosity' : -1,
            'max_depth': self.hyperparameters['max_depth'],
            'reg_alpha' : 3,
            'reg_lambda' : 3,
            'n_jobs' : n_jobs,
        }
        
        params_xgboost = {
            'random_state' : self.hyperparameters['random_state'],
            'n_estimators' : self.hyperparameters['xgb_iterations'],
            'early_stopping_rounds' : self.hyperparameters['xgb_early_stopping_rounds'],
            'learning_rate' : self.hyperparameters['xgb_learning_rate'],
            'objective' : 'reg:squarederror',#'reg:absoluteerror',
            'verbosity' : 0,
            'max_depth': self.hyperparameters['max_depth'],
            'n_jobs' : n_jobs,
        }
        
        # params_extratrees = {
        #     'random_state' : self.hyperparameters['random_state'],
        #     'n_estimators' : self.hyperparameters['extratrees_iterations'],
        #     'criterion' : 'absolute_error',
        #     'verbose' : 0,
        #     'max_depth' : self.hyperparameters['max_depth'],
        #     'n_jobs' : n_jobs,
        # }
        
        params_hgb = {
            'random_state' : self.hyperparameters['random_state'],
            'max_iter' : self.hyperparameters['hgb_iterations'],
            'verbose' : 0,
            'max_depth' : self.hyperparameters['max_depth'],
        }
        
        self.regressors = [
            # LinearRegression(),
            # RidgeCV(**params_ridge),
            # LassoCV(**params_lasso),
            # ElasticNetCV(**params_elasticnet),
            CatBoostRegressor(**params_catboost1),
            # CatBoostRegressor(**params_catboost2),
            # CatBoostRegressor(**params_catboost3),
            # CatBoostRegressor(**params_catboost4),
            MultiOutputXGBRegressor(XGBRegressor(**params_xgboost)),
            MultiOutputLGBMRegressor(LGBMRegressor(**params_lightgbm1)),
            # LGBMRegressor(**params_lightgbm2),
            # LGBMRegressor(**params_lightgbm3),
            # LGBMRegressor(**params_lightgbm4),
            # LGBMRegressor(**params_lightgbm5),
            # LGBMRegressor(**params_lightgbm6),
            # LGBMRegressor(**params_lightgbm7),
            # LGBMRegressor(**params_lightgbm8),
            # MultiOutputExtraTreesRegressor(ExtraTreesRegressor(**params_extratrees)),
            
            MultiOutputHistGradientBoostingRegressor(HistGradientBoostingRegressor(**params_hgb)),
        ]
        
        self.stacking_regressors = [
            # LinearRegression(),
            # RidgeCV(**params_ridge),
            # LassoCV(**params_lasso),
            # ElasticNetCV(**params_elasticnet),
            # CatBoostRegressor(**params_catboost1),
            # CatBoostRegressor(**params_catboost2),
            # CatBoostRegressor(**params_catboost3),
            # CatBoostRegressor(**params_catboost4),
            # XGBRegressor(**params_xgboost),
            LGBMRegressor(**params_lightgbm1),
            LGBMRegressor(**params_lightgbm2),
            LGBMRegressor(**params_lightgbm3),
            LGBMRegressor(**params_lightgbm4),
            LGBMRegressor(**params_lightgbm5),
            LGBMRegressor(**params_lightgbm6),
            LGBMRegressor(**params_lightgbm7),
            LGBMRegressor(**params_lightgbm8),
            # ExtraTreesRegressor(**params_extratrees),
        ]
        
    def _get_regressors_name(self):
        self.regressors_name = [type(r).__name__ for r in self.regressors]
        self.stacking_regressors_name = [type(r).__name__ for r in self.stacking_regressors]
        
    def _get_ohe(self,X,cat_features):
        ohe = OneHotEncoder()
        ohe.fit(X,cat_features,remove_first=False)
        return ohe
        
    def _set_zero_to_minimum(self,pred,minimum_value):
        pred = np.array(pred).flatten()
        if np.where(pred<0,1,0).sum()>0:
            pred = [x if x>0 else minimum_value for x in pred]
        pred = np.array(pred).flatten()
        return pred
    
    def _set_inf_to_maximum(self,pred,maximum_value):
        pred = np.array(pred).flatten()
        if np.where(pred==np.inf,1,0).sum()>0:
            pred = [x if x!=np.inf else maximum_value for x in pred]
        pred = np.array(pred).flatten()
        return pred
    
    def _preprocess(self,pred):
        # pred = self._set_zero_to_minimum(pred,self.minimum_value)
        # pred = self._set_inf_to_maximum(pred,self.maximum_value)
        return pred
    
    def _fit_regressor(self,
                       regressor,regressor_name,
                       X,X_oh,X_val,X_val_oh,y,y_val,cat_features,
                       sample_weight,eval_sample_weight):
        X = X[self.features]
        X_val = X_val[self.features]
        X_oh = X_oh[self.oh_features]
        X_val_oh = X_val_oh[self.oh_features]
        
        if (regressor_name in ['LinearRegression','RidgeCV','LassoCV','ElasticNetCV']) or\
            (regressor_name.find('ExtraTreesRegressor')>=0) or\
            (regressor_name.find('RandomForestRegressor')>=0) or\
            (regressor_name.find('HistGradientBoostingRegressor')>=0):
            warnings.filterwarnings("ignore", category=UserWarning)
            # fitting
            regressor.fit(X_oh,y)
            # prediction
            tr_pred = self._preprocess(regressor.predict(X_oh))
            va_pred = self._preprocess(regressor.predict(X_val_oh))

        elif regressor_name.find('XGBRegressor')>=0:
            # fitting
            regressor.fit(
                X_oh,y,
                eval_set=[(X_val_oh,y_val)],
                verbose=0,
            )
            # prediction
            tr_pred = self._preprocess(regressor.predict(X_oh))
            va_pred = self._preprocess(regressor.predict(X_val_oh))

        elif regressor_name.find('CatBoostRegressor')>=0:
            # dataset
            train_dataset = Pool(X    ,y    ,cat_features=cat_features)
            val_dataset   = Pool(X_val,y_val,cat_features=cat_features)
            # fitting
            regressor.fit(
                train_dataset,
                eval_set=val_dataset,
                #metric_period=self.hyperparameters['iterations']//50, verbose=True,
                verbose=False,
            )
            # prediction
            tr_pred = self._preprocess(regressor.predict(train_dataset))
            va_pred = self._preprocess(regressor.predict(val_dataset))

        elif regressor_name.find('LGBMRegressor')>=0:
            warnings.filterwarnings("ignore", category=UserWarning)
            # # astype category
            # for col in cat_features:
            #     X[col]     = X[col]    .astype('category')
            #     X_val[col] = X_val[col].astype('category')
            # # fitting
            # regressor.fit(
            #     X,y,
            #     eval_set=[(X_val,y_val)],
            #     sample_weight=sample_weight,
            #     eval_sample_weight=eval_sample_weight,
            #     categorical_feature=cat_features,
            #     verbose=-1,
            # )
            # fitting
            regressor.fit(
                X_oh,y,
                eval_set=[(X_val_oh,y_val)],
                sample_weight=sample_weight,
                eval_sample_weight=eval_sample_weight,
                #categorical_feature=cat_features,
                verbose=-1,
            )
            tr_pred = self._preprocess(regressor.predict(X_oh))
            va_pred = self._preprocess(regressor.predict(X_val_oh))
            
        else:
            raise ValueError('Unknown Regressor: {}'.format(regressor_name))
            
        return regressor, tr_pred, va_pred
            
    def _get_prediction_values(self,X,X_oh,method,regressors_name,regressors,weights,return_weighted):
        if method=='ensemble':
            features    = self.features
            oh_features = self.oh_features
        elif method=='stacking':
            stacking_feature = [self.stacking_feature] if isinstance(self.stacking_feature,str) else self.stacking_feature
            features    = self.features + stacking_feature
            oh_features = self.oh_features + stacking_feature
        
        # (1) 예측값생성
        pred_list = []
        for regressor_name,regressor in zip(regressors_name,regressors):
            if (regressor_name in ['LinearRegression','RidgeCV','LassoCV','ElasticNetCV','RandomForestRegressor']) or\
                (regressor_name.find('ExtraTreesRegressor')>=0) or\
                (regressor_name.find('RandomForestRegressor')>=0) or\
                (regressor_name.find('XGBRegressor')>=0) or\
                (regressor_name.find('HistGradientBoostingRegressor')>=0):
                dataset = X_oh[oh_features]
            elif regressor_name.find('CatBoostRegressor')>=0:
                dataset = Pool(X[features],cat_features=self.cat_features)
            elif regressor_name.find('LGBMRegressor')>=0:
                dataset = X_oh[oh_features]
                # dataset = X[features].copy()
                # for col in self.cat_features:
                #     dataset[col] = dataset[col].astype('category')
            else:
                raise ValueError('Unknown Regressor: {}'.format(regressor_name))
            
            y_pred = self._preprocess(regressor.predict(dataset))
            pred_list.append(y_pred)
        
        # (2) return weighted or original value
        if return_weighted:
            final_pred = []
            for pred,weight in zip(pred_list,weights):
                p = np.array(pred)*weight
                final_pred.append(p)
            final_pred = np.sum(final_pred,axis=0)
            if self.inverse_transform is not None:
                final_pred = self.inverse_transform(np.array(final_pred))
                final_pred = self._preprocess(final_pred)
        else:
            final_pred = np.array(pred_list).T
            
        return final_pred
        
    def _predict(self,X,method='stacking',return_weighted=True):
        if len(self.cat_features)>0:
            X_oh = self.ohe.transform(X)
        else:
            X_oh = X.copy()
        assert len(X)==len(X_oh), \
            "X and X_oh must be same length"
        assert method in ['ensemble','stacking'], \
            "method must be one of ['ensemble','stacking']"
        
        # (1) ensemble
        pred_list = self._get_prediction_values(
            X,X_oh,
            'ensemble',
            self.regressors_name,self.regressors,
            self.ensemble_weights,return_weighted,
        )
        
        if method=='ensemble':
            return pred_list
        
        elif method=='stacking':
            # (2) stacking
            columns = [self.stacking_feature] if isinstance(self.stacking_feature,str) else self.stacking_feature
            pred_df = pd.DataFrame(pred_list,columns=columns,index=X.index)
            
            X    = pd.concat([X   ,pred_df],axis=1)
            X_oh = pd.concat([X_oh,pred_df],axis=1)

            pred_list = self._get_prediction_values(
                X,X_oh,
                'stacking',
                self.stacking_regressors_name,self.stacking_regressors,
                self.stacking_weights,return_weighted,
            )
            return pred_list
        
    def get_feature_importance(self):
        # message_print = warnings.warn
        message_print = print
        supported_models = self.regressors_name
        
#         # feature_importances_를 지원하는 모델들
#         supported_models = ['CatBoostRegressor','XGBRegressor','LGBMRegressor','ExtraTreesRegressor',
#                             'ExtraTreesRegressor', 'RandomForestRegressor', 'HistGradientBoostingRegressor']
        
#         # 지원하지않는 모델이 있는 경우 warning message
#         not_supported_1 = len([name for name in self.regressors_name if name not in supported_models])
#         not_supported_2 = 0
#         for name in self.regressors_name:
#             for m in supported_models:
#                 k = 0 if name.find(m)>=0 else 1
#                 not_supported_2 += k
        
#         if not_supported_1 + not_supported_2 > 0:
#             message_print("not support model")

        # get weighted feature importance by using ensemble_weights
        feature_importance_df = pd.DataFrame(self.features,columns=['feature'])
        for i,(regressor,regressor_name,weight) in enumerate(zip(self.regressors,self.regressors_name,self.ensemble_weights)):
            if regressor_name in supported_models:
                feature_importance = regressor.feature_importances_
                fi_list = []
                for feature in self.features:
                    fi = feature_importance[np.where(np.array(self.features)==feature)[0]]
                    fi_list.append([feature,sum(fi)])

                imp_col = f'importance{i}'
                fi_df = pd.DataFrame(fi_list,columns=['feature',imp_col]).sort_values(imp_col,ascending=False)
                fi_df[imp_col] = 100 * fi_df[imp_col] / fi_df[imp_col].sum()
                fi_df[imp_col] *= weight

                feature_importance_df = pd.merge(feature_importance_df,fi_df,how='left',on='feature')

        feature_importance_df = feature_importance_df.fillna(0)
        feature_importance_df['importance'] = feature_importance_df.drop('feature',axis=1).sum(axis=1)
        feature_importance_df = feature_importance_df[['feature','importance']]
        
        return feature_importance_df
        
    def plot_feature_importance(self):
        feature_importance_df = self.get_feature_importance()
        feature_importance_df.sort_values('importance',ascending=True,inplace=True)
        plt.figure(figsize=(15,7))
        plt.barh(feature_importance_df['feature'],feature_importance_df['importance'])
        plt.show()
            
    def fit(self,
            X,y,eval_set,cat_features,
            sample_weight=None,eval_sample_weight=None,verbose=1):
        assert len(eval_set)==1, \
            "eval_set length must be 1. len(eval_set)={}".format(len(eval_set))
        
        if len(self.regressors)!=len(self.regressors_name):
            self._get_regressors_name()
        
        self.sample_weight = sample_weight
        self.eval_sample_weight = eval_sample_weight
        self.cat_features = cat_features
        self.enable_categorical = [True if col in cat_features else False for col in X.columns]
        
        #----------------------------------------------------------------------------------------#
        # prepare dataset
        #----------------------------------------------------------------------------------------#
        X_val, y_val = eval_set[0]
        
        del_cols = return_unique_columns(X)
        X.drop(del_cols,axis=1,inplace=True)
        X_val.drop(del_cols,axis=1,inplace=True)
        self.cat_features = list(set(cat_features)-set(del_cols))
        
        if len(self.cat_features)>0:
            self.ohe = self._get_ohe(X,cat_features)
            X_oh = self.ohe.transform(X)
            X_val_oh = self.ohe.transform(X_val)
        else:
            X_oh = X.copy()
            X_val_oh = X_val.copy()
        
        del_oh_cols = return_unique_columns(X_oh)
        X_oh.drop(del_oh_cols,axis=1,inplace=True)
        X_val_oh.drop(del_oh_cols,axis=1,inplace=True)
        
        #----------------------------------------------------------------------------------------#
        # save feature names
        #----------------------------------------------------------------------------------------#
        self.features    = X   .columns.tolist()
        self.oh_features = X_oh.columns.tolist()
        
        #----------------------------------------------------------------------------------------#
        # true value
        #----------------------------------------------------------------------------------------#
        tr_true = np.array(y)
        va_true = np.array(y_val)
        if self.inverse_transform is not None:
            tr_true = self.inverse_transform(tr_true)
            va_true = self.inverse_transform(va_true)
        
        #----------------------------------------------------------------------------------------#
        # set min,max value
        #----------------------------------------------------------------------------------------#
        self.minimum_value = min(np.nanmin(y),np.nanmin(y_val))
        self.maximum_value = max(np.nanmax(y),np.nanmax(y_val))
        
        #----------------------------------------------------------------------------------------#
        # (1) ensemble fitting
        #----------------------------------------------------------------------------------------#
        # prepare ensemble fitting
        self.ensemble_scores = []
        self.ensemble_weights = []
        self.ensemble_fitting_elapsed = []
        ensemble_pbar = zip(self.regressors_name,self.regressors)

        # fitting
        if self.method=='stacking':
            if verbose:
                print('\n########  <Step1> Ensemble  ########')
        for fit_iter,(regressor_name,regressor) in enumerate(ensemble_pbar):
            s = time.time()
                
            # fit
            regressor, tr_pred, va_pred = self._fit_regressor(
                regressor, regressor_name,
                X, X_oh, X_val, X_val_oh, y, y_val, self.cat_features,
                sample_weight, eval_sample_weight,
            )
            self.regressors[fit_iter] = regressor
            
            # progress
            if self.inverse_transform is not None:
                tr_pred = self.inverse_transform(tr_pred)
                tr_pred = self._preprocess(tr_pred)
                va_pred = self.inverse_transform(va_pred)
                va_pred = self._preprocess(va_pred)
            
            tr_score = self.eval_metric(y_pred=tr_pred,y_true=tr_true)
            va_score = self.eval_metric(y_pred=va_pred,y_true=va_true)
            
            e = time.time()
            self.ensemble_scores.append(va_score)
            self.ensemble_weights.append(1/va_score)
            self.ensemble_fitting_elapsed.append(e-s)
            
            if verbose:
                blank = ' '*(11-len(regressor_name))
                fit_progress = '[{}/{}] {}{}: loss={:.3f}, val_loss={:.3f}, elasped={:.1f}s'\
                    .format(fit_iter+1,len(self.regressors),regressor_name,blank,tr_score,va_score,e-s)
                print(fit_progress)
            
        # get weighted prediction & score
        if self.weight=='equal':
            self.ensemble_weights = np.array([1.0 for _ in self.regressors])
        self.ensemble_weights /= sum(self.ensemble_weights)
        
        tr_pred = self._predict(X,method='ensemble',return_weighted=True)
        va_pred = self._predict(X_val,method='ensemble',return_weighted=True)
        
        ## -> self.predict에서 inverse_transform 해줌
        # if self.inverse_transform is not None:
        #     tr_pred = self.inverse_transform(tr_pred)
        #     va_pred = self.inverse_transform(va_pred)
        
        ens_tr_score = self.eval_metric(y_true=tr_true,y_pred=tr_pred)
        ens_va_score = self.eval_metric(y_true=va_true,y_pred=va_pred)
        
        if verbose:
            ens_fit_progress = "<Weighted Ensemble(weight='{}')> loss={:.3f}, val_loss={:.3f}, elasped={:.1f}s"\
                .format(self.weight,ens_tr_score,ens_va_score,sum(self.ensemble_fitting_elapsed))
            print(ens_fit_progress)
        
        if self.method=='ensemble':
            self.total_score = ens_va_score
            
        elif self.method=='stacking':
            #----------------------------------------------------------------------------------------#
            # (2) stacking fitting
            #----------------------------------------------------------------------------------------#
            tr_pred = self._predict(X,method='ensemble',return_weighted=self.use_weightedsum_in_stacking)
            va_pred = self._predict(X_val,method='ensemble',return_weighted=self.use_weightedsum_in_stacking)

            columns = [self.stacking_feature] if isinstance(self.stacking_feature,str) else self.stacking_feature
            tr_pred_df = pd.DataFrame(tr_pred,columns=columns,index=X.index)
            va_pred_df = pd.DataFrame(va_pred,columns=columns,index=X_val.index)
            
            X        = pd.concat([X       ,tr_pred_df],axis=1)
            X_oh     = pd.concat([X_oh    ,tr_pred_df],axis=1)
            X_val    = pd.concat([X_val   ,va_pred_df],axis=1)
            X_val_oh = pd.concat([X_val_oh,va_pred_df],axis=1)

            # prepare stacking fitting
            self.stacking_scores = []
            self.stacking_weights = []
            self.stacking_fitting_elapsed = []

            stacking_regressors = deepcopy(self.stacking_regressors)
            stacking_pbar =  zip(self.stacking_regressors_name,stacking_regressors)

            if verbose:
                print('\n########  <Step2> Stacking  ########')
            self.stacking_regressors = []
            for fit_iter,(regressor_name,regressor) in enumerate(stacking_pbar):
                s = time.time()

                # fitting
                stacking_regressor, tr_pred, va_pred = self._fit_regressor(
                    regressor, regressor_name,
                    X, X_oh, X_val, X_val_oh, y, y_val, self.cat_features,
                    sample_weight, eval_sample_weight,
                )
                self.stacking_regressors.append(stacking_regressor)

                # progress
                if self.inverse_transform is not None:
                    tr_pred = self.inverse_transform(tr_pred)
                    tr_pred = self._preprocess(tr_pred)
                    va_pred = self.inverse_transform(va_pred)
                    va_pred = self._preprocess(va_pred)

                tr_score = self.eval_metric(y_pred=tr_pred,y_true=tr_true)
                va_score = self.eval_metric(y_pred=va_pred,y_true=va_true)

                e = time.time()
                self.stacking_scores.append(va_score)
                self.stacking_weights.append(1/va_score)
                self.stacking_fitting_elapsed.append(e-s)

                if verbose:
                    blank = ' '*(11-len(regressor_name))
                    iter_str = str(fit_iter+1).zfill(len(str(len(stacking_regressors))))
                    fit_progress = '[{}/{}] {}{}: loss={:.3f}, val_loss={:.3f}, elasped={:.1f}s'\
                        .format(iter_str,len(stacking_regressors),regressor_name,blank,tr_score,va_score,e-s)
                    print(fit_progress)

            # get weighted prediction & score
            if self.weight=='equal':
                self.stacking_weights = np.array([1.0 for _ in self.stacking_regressors])
            self.stacking_weights /= sum(self.stacking_weights)

            tr_pred = self._predict(
                X   .drop(self.stacking_feature,axis=1),
                X_oh.drop(self.stacking_feature,axis=1),
                method='stacking',
            )
            va_pred = self._predict(
                X_val   .drop(self.stacking_feature,axis=1),
                X_val_oh.drop(self.stacking_feature,axis=1),
                method='stacking',
            )

            ## -> self.predict에서 inverse_transform 해줌
            # if self.inverse_transform is not None:
            #     tr_pred = self.inverse_transform(tr_pred)
            #     va_pred = self.inverse_transform(va_pred)

            stacking_tr_score = self.eval_metric(y_true=tr_true,y_pred=tr_pred)
            stacking_va_score = self.eval_metric(y_true=va_true,y_pred=va_pred)

            if verbose:
                stacking_fit_progress = "<Weighted Stacking(weight='{}')> loss={:.3f}, val_loss={:.3f}, elasped={:.1f}s"\
                    .format(self.weight,stacking_tr_score,stacking_va_score,sum(self.stacking_fitting_elapsed))
                print(stacking_fit_progress)

            self.total_score = stacking_va_score
            
        #self.feature_importances_ = self.get_feature_importance()['importance'].values.tolist()

    def predict(self,X,method=None):
        if method is None:
            method = self.method
        if (self.method=='ensemble') & (method=='stacking'):
            raise ValueError("The training method is 'ensemble', so 'stacking' prediction is not possible")
        return self._predict(X,method=self.method,return_weighted=self.use_weightedsum_in_stacking)
        
    def save(self,path):
        save_dict = {
            'ohe' : self.ohe,
            'cat_features' : self.cat_features,
            'minimum_value' : self.minimum_value,
            'maximum_value' : self.maximum_value,
            'features' : self.features,
            'oh_features' : self.oh_features,
            'hyperparameters' : self.hyperparameters,
            'inverse_transform' : self.inverse_transform,
            'sample_weight' : self.sample_weight,
            'eval_sample_weight' : self.eval_sample_weight,
            'regressors' : self.regressors,
            'ensemble_weights' : self.ensemble_weights,
            'ensemble_fitting_elapsed' : self.ensemble_fitting_elapsed,
            'ensemble_scores' : self.ensemble_scores,
            'total_score' : self.total_score,
            #'feature_importances_' : self.feature_importances_,
        }
        if self.method=='stacking':
            additional_save_dict = {
                'stacking_regressors' : self.stacking_regressors,
                'stacking_weights' : self.stacking_weights,
                'stacking_fitting_elapsed' : self.stacking_fitting_elapsed,
                'stacking_scores' : self.stacking_scores,
            }
            save_dict = {**save_dict,**additional_save_dict}
        with open(path, 'wb') as f:
            #pickle.dump(save_dict, f, protocol=pickle.HIGHEST_PROTOCOL)
            dill.dump(save_dict, f)
            
    def load(self,path):
        with open(path, 'rb') as f:
            #save_dict = pickle.load(f)
            save_dict = dill.load(f)
            self.ohe = save_dict['ohe']
            self.cat_features = save_dict['cat_features']
            self.minimum_value = save_dict['minimum_value']
            self.maximum_value = save_dict['maximum_value']
            self.features = save_dict['features']
            self.oh_features = save_dict['oh_features']
            self.hyperparameters = save_dict['hyperparameters']
            self.inverse_transform = save_dict['inverse_transform']
            self.sample_weight = save_dict['sample_weight']
            self.eval_sample_weight = save_dict['eval_sample_weight']
            self.regressors = save_dict['regressors']
            self.ensemble_weights = save_dict['ensemble_weights']
            self.ensemble_fitting_elapsed = save_dict['ensemble_fitting_elapsed']
            self.ensemble_scores = save_dict['ensemble_scores']
            self.total_score = save_dict['total_score']
            #self.feature_importances_ = save_dict['feature_importances_']
            
            if self.method=='stacking':
                self.stacking_regressors = save_dict['stacking_regressors']
                self.stacking_weights = save_dict['stacking_weights']
                self.stacking_fitting_elapsed = save_dict['stacking_fitting_elapsed']
                self.stacking_scores = save_dict['stacking_scores']

In [23]:
class KfoldWeightedEnsembleRegressor(BaseEstimator, RegressorMixin):
    def __init__(self,
                 hyperparameters,
                 method='ensemble',
                 weight='balanced',
                 inverse_transform=None,
                 eval_metric=None,
                 use_ensemble=True,
                 n_splits=5,
                 random_state=0):
        super().__init__()
        
        assert weight in ['equal','balanced'], \
            "weight must be one of ['equal','balanced']"
        
        self.hyperparameters = hyperparameters
        self.method = method
        self.weight = weight
        self.inverse_transform = inverse_transform
        self.eval_metric = RMSE if eval_metric is None else eval_metric
        self.use_ensemble = use_ensemble
        self.n_splits = n_splits
        self.random_state = random_state
        
    def get_feature_importance(self):
        fs = [m.features for m in self.base_models]
        fs = list(set(item for sublist in fs for item in sublist))
        feature_importance_df = pd.DataFrame(fs,columns=['feature'])

        for i,(base_model,base_feature_importance) in enumerate(zip(self.base_models,self.base_feature_importances)):
            imp_col = f'imp{i}'
            imp_df = pd.DataFrame({
                'feature' : base_model.features,
                imp_col : base_feature_importance,
            })
            feature_importance_df = pd.merge(feature_importance_df,imp_df,how='left',on='feature')

        feature_importance_df.fillna(0,inplace=True)
        feature_importance_df['importance'] = feature_importance_df.drop('feature',axis=1).sum(axis=1)
        feature_importance_df['importance'] = 100 * feature_importance_df['importance'] / feature_importance_df['importance'].sum()
        
        return feature_importance_df
        
    def plot_feature_importance(self):
        feature_importance_df = self.get_feature_importance()
        feature_importance_df.sort_values('importance',ascending=True,inplace=True)

        plt.figure(figsize=(15,7))
        plt.barh(feature_importance_df.feature,feature_importance_df.importance)
        plt.show()
        
    def fit(self,X,y,cat_features,sample_weight=None,verbose=True):
        self.cat_features = cat_features
        self.sample_weight = sample_weight
        self.features = X.columns.tolist()

        self.base_models = []
        self.base_scores = []
        self.base_feature_importances = []
        kf = KFold(n_splits=self.n_splits,random_state=self.random_state,shuffle=True)

        progress_fmt = '> KFold: {}/{}'
        for k, (tr_idx, val_idx) in enumerate(kf.split(X)):
            k_str = str(k+1).zfill(len(str(self.n_splits)))
            print('')
            print('-'*80)
            print(progress_fmt.format(k_str,self.n_splits))
            print('-'*80)
            
            X_tr   , X_va    = X   .iloc[tr_idx], X   .iloc[val_idx]
            y_tr   , y_va    = y   .iloc[tr_idx], y   .iloc[val_idx]
            
            if self.sample_weight is None:
                sample_weight = None
                eval_sample_weight = None
            else:
                sample_weight = self.sample_weight[tr_idx]
                eval_sample_weight = self.sample_weight[val_idx]

            #------------------------------------------------------------------------------------#
            # (1) base model
            #------------------------------------------------------------------------------------#
            # define the base model
            base_model = WeightedEnsembleRegressor(
                hyperparameters,
                weight='balanced', # 'equal', 'balanced',
                inverse_transform=self.inverse_transform,
                eval_metric=self.eval_metric,
                method=self.method, # 'ensemble', 'stacking'
                use_weightedsum_in_stacking=True,
            )
            # fit the model
            base_model.fit(
                X_tr,y_tr,
                eval_set=[(X_va,y_va)],
                cat_features=cat_features,
                sample_weight=sample_weight,
                eval_sample_weight=[eval_sample_weight],
                verbose=verbose,
            )
            
            # prediction
            y_pred = base_model.predict(X_va)
            if self.inverse_transform is not None:
                y_true = self.inverse_transform(y_va.values)
            else:
                y_true = y_va.values
            
            # caculate score
            score = mean_squared_error(y_true=y_true,y_pred=y_pred)**0.5

            # append inner loop
            self.base_models.append(base_model)
            self.base_scores.append([k+1,len(X_tr),len(X_va),score])
            
            # # plot feature importance
            # self.base_feature_importances.append(base_model.feature_importances_)
            # base_model.plot_feature_importance()
        
        self.base_score = pd.DataFrame(self.base_scores,columns=['k','n_train','n_val','rmse'])
        self.validation_score = self.base_score.rmse.mean()
        
        #self.plot_feature_importance()
        
    def predict(self,X):
        pred = [base_model.predict(X) for base_model in self.base_models]
        pred = np.mean(pred,axis=0)
        return pred
    
    def save(self,path):
        save_dict = {
            'hyperparameters' : self.hyperparameters,
            'weight' : self.weight,
            'n_splits' : self.n_splits,
            'random_state' : self.random_state,
            'inverse_transform' : self.inverse_transform,
            'cat_features' : self.cat_features,
            'sample_weight' : self.sample_weight,
            'base_models' : self.base_models,
            'base_scores' : self.base_scores,
            'base_score' : self.base_score,
            'validation_score' : self.validation_score,
            'base_feature_importances' : self.base_feature_importances,
        }
        with open(path, 'wb') as f:
            #pickle.dump(save_dict, f, protocol=pickle.HIGHEST_PROTOCOL)
            dill.dump(save_dict, f)
            
    def load(self,path):
        with open(path, 'rb') as f:
            #save_dict = pickle.load(f)
            save_dict = dill.load(f)
            
            self.hyperparameters = save_dict['hyperparameters']
            self.weight = save_dict['weight']
            self.n_splits = save_dict['n_splits']
            self.random_state = save_dict['random_state']
            self.inverse_transform = save_dict['inverse_transform']
            self.cat_features = save_dict['cat_features']
            self.sample_weight = save_dict['sample_weight']
            self.base_models = save_dict['base_models']
            self.base_scores = save_dict['base_scores']
            self.base_score = save_dict['base_score']
            self.validation_score = save_dict['validation_score']
            self.base_feature_importances = save_dict['base_feature_importances']

In [31]:
hyperparameters = {
    'max_depth' : 9,
    'random_state' : CFG.SEED,
    'learning_rate' : 0.01,
    'iterations' : 30000,
    'early_stopping_rounds' : 300,
    'xgb_learning_rate' : 0.3,         # default=0.3
    'xgb_iterations' : 300,           # default=100
    'xgb_early_stopping_rounds' : 30,
    # 'extratrees_iterations' : 5,#100,     # default=100
    'hgb_iterations' : 3, # default=100
}

In [32]:
eclo_weight = [10,5,3,1]
eval_metric = lambda y_true,y_pred: WeightedMultiRMSE(y_true,y_pred,weight=eclo_weight)

mc_path_fmt = './mc/weiens_multirmse_seg_gungu/{}.pkl'
mkdir('/'.join(mc_path_fmt.split('/')[:-1]))

In [33]:
%%time

gungu_list = train_df['gungu'].unique()
models = []
for gungu in tqdm(gungu_list):
    train_data = train_df[train_df.gungu==gungu].drop('gungu',axis=1)
    test_data  = test_df [test_df .gungu==gungu].drop('gungu',axis=1)
    
    X = train_data.drop(CFG.TARGET,axis=1)
    y = train_data[CFG.TARGET]
    fixed_cat_features = setdiff(cat_features,['gungu'])
    
    model = KfoldWeightedEnsembleRegressor(
        hyperparameters,
        method='ensemble',
        weight='balanced',
        inverse_transform=target_transform.inverse_transform,
        eval_metric=eval_metric,
        use_ensemble=True,
        n_splits=CFG.N_SPLITS,
        random_state=CFG.SEED,
    )
    model.fit(
        X,y,
        cat_features=fixed_cat_features,
        sample_weight=None,
        verbose=True,
    )
    model.save(mc_path_fmt.format(gungu))

  0%|                                                     | 0/8 [00:00<?, ?it/s]


--------------------------------------------------------------------------------
> KFold: 1/5
--------------------------------------------------------------------------------
[1/4] CatBoostRegressor: loss=0.350, val_loss=0.360, elasped=37.4s
[2/4] MultiOutputXGBRegressor: loss=0.261, val_loss=0.378, elasped=56.1s
[3/4] MultiOutputLGBMRegressor: loss=0.326, val_loss=0.360, elasped=12.5s
[4/4] MultiOutputHistGradientBoostingRegressor: loss=0.350, val_loss=0.363, elasped=9.8s
<Weighted Ensemble(weight='balanced')> loss=0.318, val_loss=0.361, elasped=115.9s

--------------------------------------------------------------------------------
> KFold: 2/5
--------------------------------------------------------------------------------
[1/4] CatBoostRegressor: loss=0.348, val_loss=0.346, elasped=42.7s
[2/4] MultiOutputXGBRegressor: loss=0.266, val_loss=0.370, elasped=49.4s
[3/4] MultiOutputLGBMRegressor: loss=0.328, val_loss=0.346, elasped=12.0s
[4/4] MultiOutputHistGradientBoostingRegressor: l

 12%|█████▌                                      | 1/8 [08:04<56:28, 484.01s/it]


--------------------------------------------------------------------------------
> KFold: 1/5
--------------------------------------------------------------------------------
[1/4] CatBoostRegressor: loss=0.314, val_loss=0.331, elasped=89.8s
[2/4] MultiOutputXGBRegressor: loss=0.258, val_loss=0.341, elasped=27.1s
[3/4] MultiOutputLGBMRegressor: loss=0.306, val_loss=0.331, elasped=41.7s
[4/4] MultiOutputHistGradientBoostingRegressor: loss=0.321, val_loss=0.335, elasped=5.2s
<Weighted Ensemble(weight='balanced')> loss=0.297, val_loss=0.331, elasped=163.8s

--------------------------------------------------------------------------------
> KFold: 2/5
--------------------------------------------------------------------------------
[1/4] CatBoostRegressor: loss=0.319, val_loss=0.318, elasped=189.2s
[2/4] MultiOutputXGBRegressor: loss=0.265, val_loss=0.338, elasped=29.0s
[3/4] MultiOutputLGBMRegressor: loss=0.314, val_loss=0.319, elasped=32.0s
[4/4] MultiOutputHistGradientBoostingRegressor: 

 25%|██████████▌                               | 2/8 [30:25<1:38:50, 988.41s/it]

...


--------------------------------------------------------------------------------
> KFold: 1/5
--------------------------------------------------------------------------------
[1/4] CatBoostRegressor: loss=0.390, val_loss=0.389, elasped=8.3s
[2/4] MultiOutputXGBRegressor: loss=0.314, val_loss=0.418, elasped=1.1s
[3/4] MultiOutputLGBMRegressor: loss=0.385, val_loss=0.391, elasped=12.0s
[4/4] MultiOutputHistGradientBoostingRegressor: loss=0.394, val_loss=0.392, elasped=0.5s
<Weighted Ensemble(weight='balanced')> loss=0.367, val_loss=0.391, elasped=21.7s

--------------------------------------------------------------------------------
> KFold: 2/5
--------------------------------------------------------------------------------
[1/4] CatBoostRegressor: loss=0.393, val_loss=0.391, elasped=5.8s
[2/4] MultiOutputXGBRegressor: loss=0.318, val_loss=0.433, elasped=1.1s
[3/4] MultiOutputLGBMRegressor: loss=0.381, val_loss=0.392, elasped=11.3s
[4/4] MultiOutputHistGradientBoostingRegressor: loss=0

 88%|██████████████████████████████████████▌     | 7/8 [44:06<03:46, 226.89s/it]


--------------------------------------------------------------------------------
> KFold: 1/5
--------------------------------------------------------------------------------
[1/4] CatBoostRegressor: loss=0.315, val_loss=0.346, elasped=12.3s
[2/4] MultiOutputXGBRegressor: loss=0.233, val_loss=0.362, elasped=0.9s
[3/4] MultiOutputLGBMRegressor: loss=0.306, val_loss=0.346, elasped=12.6s
[4/4] MultiOutputHistGradientBoostingRegressor: loss=0.323, val_loss=0.351, elasped=0.4s
<Weighted Ensemble(weight='balanced')> loss=0.290, val_loss=0.346, elasped=26.2s

--------------------------------------------------------------------------------
> KFold: 2/5
--------------------------------------------------------------------------------
[1/4] CatBoostRegressor: loss=0.329, val_loss=0.318, elasped=6.1s
[2/4] MultiOutputXGBRegressor: loss=0.275, val_loss=0.345, elasped=0.9s
[3/4] MultiOutputLGBMRegressor: loss=0.313, val_loss=0.318, elasped=16.5s
[4/4] MultiOutputHistGradientBoostingRegressor: loss=

100%|████████████████████████████████████████████| 8/8 [46:40<00:00, 350.10s/it]

CPU times: user 48min 29s, sys: 21min 12s, total: 1h 9min 41s
Wall time: 46min 40s


In [27]:
# feature_importance_df = model.get_feature_importance()
# feature_importance_df.sort_values('importance',ascending=True,inplace=True)
# feature_importance_df = feature_importance_df.tail(10)

# plt.figure(figsize=(15,7))
# plt.barh(feature_importance_df.feature,feature_importance_df.importance)
# plt.show()

In [40]:
tr_preds = []
te_preds = []
for gungu in tqdm(gungu_list):
    train_data = train_df[train_df.gungu==gungu].drop('gungu',axis=1)
    test_data  = test_df [test_df .gungu==gungu].drop('gungu',axis=1)
    
    X = train_data.drop(CFG.TARGET,axis=1)
    y = train_data[CFG.TARGET]
    fixed_cat_features = setdiff(cat_features,['gungu'])
    
    model = KfoldWeightedEnsembleRegressor(
        hyperparameters,
        method='ensemble',
        weight='balanced',
        inverse_transform=target_transform.inverse_transform,
        eval_metric=eval_metric,
        use_ensemble=True,
        n_splits=CFG.N_SPLITS,
        random_state=CFG.SEED,
    )
    model.load(mc_path_fmt.format(gungu))
    
    tr_p = model.predict(X)
    te_p = model.predict(test_data)
    
    tr_pred = pd.DataFrame(y,index=train_data.index)
    te_pred = pd.DataFrame(index=test_data.index)
    
    for i in range(len(CFG.TARGET)):
        tr_pred[f'pred_{CFG.TARGET[i]}'] = tr_p[:,i]
        te_pred[f'pred_{CFG.TARGET[i]}'] = te_p[:,i]
    
    tr_preds.append(tr_pred)
    te_preds.append(te_pred)

100%|█████████████████████████████████████████████| 8/8 [00:08<00:00,  1.05s/it]


In [44]:
tr_prediction = pd.concat(tr_preds,axis=0).sort_index()
te_prediction = pd.concat(te_preds,axis=0).sort_index()

In [57]:
WeightedMultiRMSE(y_true=tr_prediction[CFG.TARGET],
                  y_pred=tr_prediction[[f'pred_{t}' for t in CFG.TARGET]],
                  weight=eclo_weight)

0.32612910080214136

In [58]:
true = tr_prediction[CFG.TARGET] @ eclo_weight
pred = tr_prediction[[f'pred_{t}' for t in CFG.TARGET]] @ eclo_weight

RMSLE(y_true=true,y_pred=pred)

0.4321323696883392

In [65]:
submit = pd.read_csv('./data/sample_submission.csv')
submit['ECLO'] = te_prediction[[f'pred_{t}' for t in CFG.TARGET]] @ eclo_weight
submit.to_csv('./out/5_weiens_multireg_weightedmultirmse_seg(gungu).csv',index=False)
submit.head()

,ID,ECLO
0,ACCIDENT_39609,4.300596
1,ACCIDENT_39610,4.273892
2,ACCIDENT_39611,6.077025
3,ACCIDENT_39612,5.709065
4,ACCIDENT_39613,5.031157
